#cose da fare

2. discutere sulla differenza tra la trasformazione logaritmica e non dei valori azionari
3. aggiungi misure per la validazione e discutile. ad esempio MAE, MAPE e r-squared
4. discutere i coefficienti di regressione del modello scelto ( è lineare quindi parecchio semplice)
5. discuti collinearità
6. test normalità degli errori e test eteroschedasticità (?)"L'ipotesi alla base del test della normalità degli errori è una delle ipotesi classiche della regressione lineare, nota come l'ipotesi di normalità degli errori." 
" Questa è una delle quattro ipotesi principali della regressione lineare:

Linearità: La relazione tra le variabili indipendenti e la variabile dipendente è lineare.
Indipendenza degli errori: Gli errori non sono correlati tra loro.
Normalità degli errori: Gli errori seguono una distribuzione normale.
Omoschedasticità: La varianza degli errori è costante (omoschedasticità)."

In [67]:
# essential imports
import pandas as pd
import re
import yahooquery as yq
import datetime
import numpy as np

# imports for the models
import scipy.stats as st
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
import plotly.express as px


In [ ]:
# Leggi il file CSV
df = pd.read_csv('./data/tesi28.csv', sep=';')
#pulisci i nome dei df
# sostituisci gli spazi con _ e togli le parentesi
df.columns = [re.sub(r'\(.*\)', '', col) for col in df.columns]
# Rimuovere lo spazio dagli header delle colonne alla fine e tra le parole
df.columns = df.columns.str.rstrip(' ')
df.columns = df.columns.str.rstrip('.')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('/', '_')
df.columns = df.columns.str.replace('.', '')
df

## scarica i valori di borsa per ogni ticker, per ogni anno

In [ ]:
tickers = df['Ticker_symbol']
valori_borsa = []
tickers_non_scaricati=[]

# Scarica i dati storici per ciascun ticker e anno
for ticker in tickers:
    yq_ticker = yq.Ticker(ticker+".mi")
    for year in range(2023, 2013, -1):
        # Specifica la data desiderata come "YYYY-MM-DD" (31 dicembre dell'anno specificato)
        datainizio = f"{year}-01-01"
        datafine = f"{year}-01-10"

        # Converti la data desiderata in un oggetto datetime
        datainizio = datetime.datetime.strptime(datainizio, "%Y-%m-%d").date()
        datafine = datetime.datetime.strptime(datafine, "%Y-%m-%d").date()


        try:
            val = yq_ticker.history(start=datainizio, end=datafine)['close'][-1]
        except Exception as e:
            print(e.__cause__)
            val = "NaN"
            tickers_non_scaricati.append(ticker)

        print(ticker, val)
        valori_borsa.append(val)



In [ ]:
print(len(valori_borsa))
tickers_non_scaricati = set(tickers_non_scaricati)
print(tickers_non_scaricati)

## crea nuovo df
1. prendi info di interesse, moltiplica per ogni anno ogni osservazione 
2. aggiungi gli anni di interesse
3. doppio ciclo for: per ogni riga df originale prendi i primi nove valori e trasponi il risultato, passa ai prossimi nove (secondo ciclo)

#### crea df delle info 

In [ ]:
info = pd.concat([df.iloc[:, 0:5]] * 10)
info.sort_index(inplace=True)
info

#### crea colonna degli anni, aggiungi a info

In [ ]:
anni = [anno for anno in range(2022, 2012, -1)] * 28
info['Anni'] = anni
info

#### df degli indici

In [ ]:
# Rimuovi il carattere '\n' e tutti quelli che seguono dalle intestazioni delle colonne
df.columns = df.columns.str.replace(r'\n.*', '', regex=True)
df.columns = df.columns.str.rstrip('_')
nomi_colonne = df.columns.unique()[5:]
# prendi df di interesse, serve a semplificare gli indici
indici = df.iloc[:, 5:]
for i in indici.columns:
    print(i)
# crea nuovo df
new_df = pd.DataFrame()

col = 0
#per ogni set di 10 valori riga di df originario INDICI
for i in range(0, 140, 10):
    
    indice = []
    #per ogni riga AZIENDA
    for j in range(1,len(indici)+1):
        # prendi i nove valori, trasponili e aggiungili al vettore dell'indice considerato
        valori = indici.iloc[j-1:j, i:i+10].values[0]
        valori = valori.T.tolist()
        
        indice= indice + valori
        
    # aggiungi colonna a new_df con insert
    new_df.insert(col, nomi_colonne[col], indice)
    col += 1


for col in new_df.columns:
    new_df[col] = new_df[col].str.replace(',' , '.').astype(float)
    #print(type(X[col]))

new_df

#### aggiungi anche valori azionari

In [ ]:
# aggiungi i valori della borsa al df info
info['Valori_Borsa'] = valori_borsa
info

##### esegui il join dei due dataframe, resetta gli indici!!


In [ ]:
#resetta indici
info = info.reset_index(drop=True)
new_df = new_df.reset_index(drop=True)
#esegui il join
dati_puliti = info.join(new_df)
dati_puliti


### ESEGUI SE NON VUOI AZIENDE SENZA VALORI DI BORSA
##### ripulisci i dati, togli le aziende che non hanno valori di borsa per almeno un anno e calcola gli indici per ogni azienda
i valori di borsa non sono presenti in quanto alcune di queste aziende non erano quotate durante il periodo di analisi.

In [ ]:
# elimina le aziende tramite una maschera
tickers_non_scaricati = list(set(tickers_non_scaricati))
maschera = dati_puliti['Ticker_symbol'].isin(tickers_non_scaricati)
dati_puliti = dati_puliti[~maschera]
#df_pulito = df_pulito.drop(columns=["Unnamed: 0"])
dati_puliti

### ESEGUI PER SALVARE DF DELLE AZIENDE CHE NON HANNO ALCUNI VALORI DI BORSA
DF È PIÙ PICCOLO

In [ ]:
dati_puliti.to_csv("./data/dati_puliti.csv")

### ESEGUI PER SALVARE DF DI TUTTE LE AZIENDE

In [ ]:
dati_puliti.to_csv("./data/dati_puliti28.csv")

# CREA IL MODELLO
attenzione al path usato, esistono due file: uno con tutte le aziende (dati_puliti28) e uno con solo le aziende che contengono valori di borsa(dati_puliti)

In [109]:
df = pd.read_csv('./data/dati_puliti28.csv', sep=',')
df = df.drop(columns=["Unnamed: 0"])
#calcola i logaritmo dei valori di borsa
df['Valori_Borsa'] = np.log(df['Valori_Borsa'])
df

,ID,Ragione_sociale,Ticker_symbol,ISIN_number,Numero_CCIAA,Anni,Valori_Borsa,Indice_corrente,Indice_di_liquidità,Rotaz_cap_investito,Rotaz_cap_cir_lordo,Redditività_del_totale_attivo,Redditività_di_tutto_il_capitale_investito,Redditività_delle_vendite,Redditività_del_capitale_proprio,Indice_di_copertura_delle_immob,Costo_denaro_a_prestito,Oneri_finanz_su_fatt,Grado_di_copertura_degli_interessi_passivi,Debiti_v_banche_su_fatt,Debt_Equity_ratio
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,2.509599,1.58,1.18,1.55,2.21,3.44,5.34,2.22,7.77,0.45,3.79,0.50,8.11,13.25,0.95
1,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,2.934920,1.57,1.20,1.17,1.66,4.61,7.26,3.83,9.50,0.42,4.31,0.68,8.88,15.76,0.86
2,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,2.879198,1.70,1.38,0.98,1.40,0.09,0.12,0.09,-1.25,0.37,1.60,0.57,5.90,35.37,1.29
3,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,3.054001,1.46,1.16,1.46,2.04,8.84,13.31,5.88,19.42,0.33,2.15,0.40,19.02,18.59,1.04
4,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,3.068983,1.67,1.34,1.54,2.06,9.37,14.36,5.92,20.32,0.20,1.87,0.35,20.30,18.72,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,1.324419,3.34,3.31,0.02,0.04,6.48,7.48,22.97,5.28,0.03,1.42,1.13,25.64,79.55,0.35
276,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,1.200567,2.39,2.37,0.02,0.03,6.51,8.08,24.26,8.55,0.02,2.01,1.26,25.00,62.68,0.27
277,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,1.468335,2.55,2.53,0.02,0.03,7.92,9.94,29.06,13.71,0.02,3.10,1.53,23.10,49.29,0.20
278,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,0.816249,2.23,2.21,0.02,0.03,7.20,9.28,25.64,12.34,0.02,9.10,2.33,13.82,25.64,0.10


##### mostra le correlazioni tra i valori di borsa e tutti gli indici

In [57]:
#print(df.columns[6:])
correlations = pd.DataFrame(df.iloc[ : , 6:].corr().iloc[0, 1:])
correlations

,Valori_Borsa
Indice_corrente,0.066578
Indice_di_liquidità,0.074710
Rotaz_cap_investito,0.039606
Rotaz_cap_cir_lordo,-0.222344
Redditività_del_totale_attivo,0.444090
Redditività_di_tutto_il_capitale_investito,0.424775
Redditività_delle_vendite,0.288598
Redditività_del_capitale_proprio,0.431829
Indice_di_copertura_delle_immob,-0.354884
Costo_denaro_a_prestito,-0.226434


In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Indice_corrente", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Indice_di_liquidità", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Rotaz_cap_investito", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Rotaz_cap_cir_lordo", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_del_totale_attivo", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_di_tutto_il_capitale_investito", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_delle_vendite", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_del_capitale_proprio", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Indice_di_copertura_delle_immob", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Costo_denaro_a_prestito", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Oneri_finanz_su_fatt", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Grado_di_copertura_degli_interessi_passivi", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Debiti_v_banche_su_fatt", color='Ticker_symbol')
fig

In [ ]:
fig = px.scatter(df, y="Valori_Borsa", x="Debt_Equity_ratio", color='Ticker_symbol')
fig

# prepara il df

In [110]:
# crea le variabili dummy
df_dummies = pd.get_dummies(df["Ticker_symbol"])
print(df_dummies)
# unsci le dummies al df
df_con_dummies = df.join(df_dummies)
df_con_dummies


       ALA    BAN     BC    BRE    CFL    CLI    DAN    ELC    ENV    FCM  \
0    False  False  False  False  False  False  False  False  False  False   
1    False  False  False  False  False  False  False  False  False  False   
2    False  False  False  False  False  False  False  False  False  False   
3    False  False  False  False  False  False  False  False  False  False   
4    False  False  False  False  False  False  False  False  False  False   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
275  False   True  False  False  False  False  False  False  False  False   
276  False   True  False  False  False  False  False  False  False  False   
277  False   True  False  False  False  False  False  False  False  False   
278  False   True  False  False  False  False  False  False  False  False   
279  False   True  False  False  False  False  False  False  False  False   

       FOS    FVI    GPI    ICF    ILP    ITD    LDB   MARP   MARR   MASI  

,ID,Ragione_sociale,Ticker_symbol,ISIN_number,Numero_CCIAA,Anni,Valori_Borsa,Indice_corrente,Indice_di_liquidità,Rotaz_cap_investito,Rotaz_cap_cir_lordo,Redditività_del_totale_attivo,Redditività_di_tutto_il_capitale_investito,Redditività_delle_vendite,Redditività_del_capitale_proprio,Indice_di_copertura_delle_immob,Costo_denaro_a_prestito,Oneri_finanz_su_fatt,Grado_di_copertura_degli_interessi_passivi,Debiti_v_banche_su_fatt,Debt_Equity_ratio,ALA,BAN,BC,BRE,CFL,CLI,DAN,ELC,ENV,FCM,FOS,FVI,GPI,ICF,ILP,ITD,LDB,MARP,MARR,MASI,MDC,PIA,PLT,SCK,SRI,STAR7,SVS,VIM
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,2.509599,1.58,1.18,1.55,2.21,3.44,5.34,2.22,7.77,0.45,3.79,0.50,8.11,13.25,0.95,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,2.934920,1.57,1.20,1.17,1.66,4.61,7.26,3.83,9.50,0.42,4.31,0.68,8.88,15.76,0.86,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
2,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,2.879198,1.70,1.38,0.98,1.40,0.09,0.12,0.09,-1.25,0.37,1.60,0.57,5.90,35.37,1.29,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,3.054001,1.46,1.16,1.46,2.04,8.84,13.31,5.88,19.42,0.33,2.15,0.40,19.02,18.59,1.04,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,3.068983,1.67,1.34,1.54,2.06,9.37,14.36,5.92,20.32,0.20,1.87,0.35,20.30,18.72,0.94,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,1.324419,3.34,3.31,0.02,0.04,6.48,7.48,22.97,5.28,0.03,1.42,1.13,25.64,79.55,0.35,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
276,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,1.200567,2.39,2.37,0.02,0.03,6.51,8.08,24.26,8.55,0.02,2.01,1.26,25.00,62.68,0.27,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
277,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,1.468335,2.55,2.53,0.02,0.03,7.92,9.94,29.06,13.71,0.02,3.10,1.53,23.10,49.29,0.20,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
278,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,0.816249,2.23,2.21,0.02,0.03,7.20,9.28,25.64,12.34,0.02,9.10,2.33,13.82,25.64,0.10,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [111]:
df_con_dummies.columns

Index(['ID', 'Ragione_sociale', 'Ticker_symbol', 'ISIN_number', 'Numero_CCIAA',
       'Anni', 'Valori_Borsa', 'Indice_corrente', 'Indice_di_liquidità',
       'Rotaz_cap_investito', 'Rotaz_cap_cir_lordo',
       'Redditività_del_totale_attivo',
       'Redditività_di_tutto_il_capitale_investito',
       'Redditività_delle_vendite', 'Redditività_del_capitale_proprio',
       'Indice_di_copertura_delle_immob', 'Costo_denaro_a_prestito',
       'Oneri_finanz_su_fatt', 'Grado_di_copertura_degli_interessi_passivi',
       'Debiti_v_banche_su_fatt', 'Debt_Equity_ratio', 'ALA', 'BAN', 'BC',
       'BRE', 'CFL', 'CLI', 'DAN', 'ELC', 'ENV', 'FCM', 'FOS', 'FVI', 'GPI',
       'ICF', 'ILP', 'ITD', 'LDB', 'MARP', 'MARR', 'MASI', 'MDC', 'PIA', 'PLT',
       'SCK', 'SRI', 'STAR7', 'SVS', 'VIM'],
      dtype='object')

Construct the regression equation. Note that we are leaving out one dummy variable so as to avoid perfect Multicollinearity between the 7 dummy variables. la dummy omessa è BAN 
#RIVEDI

### definisci le variabili da usare nel modello

In [129]:
# definisce le var dipendenti e indipendenti
y_var_name = 'Valori_Borsa'
x_var_names =  [
    'Anni',
    #'Indice_corrente', 
    'Indice_di_liquidità',
    #'Rotaz_cap_investito', 
    #'Rotaz_cap_cir_lordo',
    #'Redditività_del_totale_attivo',
    #'Redditività_di_tutto_il_capitale_investito',
    'Redditività_delle_vendite', 
    #'Redditività_del_capitale_proprio',
    #'Indice_di_copertura_delle_immob', 
    #'Costo_denaro_a_prestito',
    #'Oneri_finanz_su_fatt', 
    'Grado_di_copertura_degli_interessi_passivi',
    #'Debiti_v_banche_su_fatt', 
    #'Debt_Equity_ratio'
    ]

unit_names = [
    'MARR', 
    'PIA', 
    'BRE', 
    'DAN', 
    'BC', 
    'CLI', 
    'ELC', 
    'GPI', 
    'SRI',
    'ALA', 
    'ICF', 
    'ENV', 
    'MASI', 
    'SVS', 
    'STAR7', 
    'SCK', 
    'MARP', 
    'ITD',
    'VIM', 
    'MDC', 
    'ILP', 
    'CFL', 
    'FCM', 
    'FVI', 
    'PLT', 
    'LDB', 
    'FOS',
    'BAN'
    ]

#only_value = [x for x in unit_names if x not in tickers_non_scaricati]
#unit_names = only_value


### crea il modello

In [130]:

lsdv_expr = y_var_name + ' ~ '
i = 0
for x_var_name in x_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + x_var_name
    else:
        lsdv_expr = lsdv_expr + x_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name
 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=Valori_Borsa ~ Anni + Indice_di_liquidità + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS


In [131]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_con_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:           Valori_Borsa   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     41.73
Date:                Tue, 12 Sep 2023   Prob (F-statistic):           2.51e-54
Time:                        14:23:27   Log-Likelihood:                -31.824
No. Observations:                 166   AIC:                             125.6
Df Residuals:                     135   BIC:                             222.1
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

### previsioni

In [65]:
errors = []

In [132]:
# Ottieni le previsioni per tutte le righe nel dataframe
predictions = lsdv_model_results.predict()

# trasform predictions to the original scale and the share values
predictions = np.e**predictions
val_borsa = (np.e**df['Valori_Borsa']).dropna()
#val_borsa = df['Valori_Borsa'].dropna()

rmse = np.sqrt(mean_squared_error(val_borsa, predictions))
mae = mean_absolute_error(val_borsa, predictions)
rsquared = lsdv_model_results.rsquared

# Aggiungi i dati alla lista
errors.append({
    'Formula': 'log ' + lsdv_expr,
    'RMSE': rmse,
    'MAE': mae,
    'R-Squared': rsquared,
})


In [134]:
pd.set_option('display.max_colwidth', 2500)
errors_df = pd.DataFrame(errors)
errors_df

,Formula,RMSE,MAE,R-Squared
0,Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Indice_di_copertura_delle_immob + Costo_denaro_a_prestito + Oneri_finanz_su_fatt + Grado_di_copertura_degli_interessi_passivi + Debiti_v_banche_su_fatt + Debt_Equity_ratio + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.195747,2.523982,0.823291
1,Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.243458,2.510691,0.819250
2,Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + Debiti_v_banche_su_fatt + Debt_Equity_ratio + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.201027,2.534304,0.822846
3,Valori_Borsa ~ Anni + Indice_corrente + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.323152,2.538464,0.812397
4,log Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Indice_di_copertura_delle_immob + Costo_denaro_a_prestito + Oneri_finanz_su_fatt + Grado_di_copertura_degli_interessi_passivi + Debiti_v_banche_su_fatt + Debt_Equity_ratio + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,3.554833,1.829362,0.910533
5,log Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,3.685501,1.870647,0.907196
6,log Valori_Borsa ~ Anni + Indice_di_liquidità + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.093706,1.969076,0.902654
